In [ ]:
import numpy as np
import data_pipes

df_path = "processed_aapl_data.csv"
data = data_pipes.process_df_3(df_path)
x, y, close = data["x"], data["y"], data["close"]
y = (y - y.min()) / (y.max() - y.min())
y = y - y.mean()
print(f"""
x shape: {x.shape}
y shape: {y.shape}     
close shape: {close.shape} 
""")

In [ ]:
import models

model_config = {
    'neurons': [128, 256, 256, 128, 32, 1],
    'activations': ['none', 'selu', 'selu', 'selu', 'selu', 'tanh'],
    'dropouts': [0.3, 0.3, 0.3, 0.3, 0.3, 0.3]
}

model = models.georgia_1(model_config)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x_tns = torch.tensor(x, dtype=torch.float32).to(device)
y_tns = torch.tensor(y, dtype=torch.float32).to(device)

model

In [ ]:
import torch_ops
from torch import nn, optim

loss_fn = nn.L1Loss()
optimizer = optim.NAdam(
    params=model.parameters(),
    lr=1e-5,
    # weight_decay=l2_decay
    )
batch_size = 32

trade_win = 30

for win in range(trade_win, len(x_tns)):
    x_train = x_tns[win - trade_win:win]
    y_train = y_tns[win - trade_win:win]
    x_play = x_tns[win:win + 1]

    loss = torch_ops.train_loop(x_train, y_train, model, loss_fn, optimizer, batch_size)

    if win % 10 == 0:
        print(f"win: {win}, loss: {loss}")

    

In [ ]:
5%5
